## Assignment #2 Part 3: Semantic Segmentation using FCN

Copyright (C) Data Science & Artificial Intelligence Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. June 2019

In this part, you will implement Fully Convolutional Networks(FCNs)[3] to perform semantic segmentation on Kitti-road dataset. <br>

Most of coes are completed for your convenience except FCN model.

There is **1 section**, and you need to follow the instructions to complete the skeleton codes.


## Set hyperparameters and load datasets
The datasets in the *Utils* directory will be loaded automatically. <br>

In [ ]:
import argparse
import os.path
import tensorflow as tf
import numpy as np
from Utils.batch import gen_batch_function
import tensorflow.contrib.slim as slim
import matplotlib.pyplot as plt

from PIL import Image

from Utils.fcn import FcnModel
from Utils.data_utils import maybe_download_and_extract, load_kitti_road, save_test_samples

import os

os.environ["CUDA_VISIBLE_DEVICES"]='2'

dataset = './data'
test_output_dir = './test_run'
model = "models/model.ckpt"
epochs = 20
batch = 64
lr = 1e-2

image_shape = (576, 160)
num_classes = 2

# Create function to get batches
get_batches_fn = load_kitti_road(image_shape)
 

## Kitti Road Dataset Sample
### data <img src="Utils/um_000000.png" alt="drawing" width="400"/>
### label <img src="Utils/um_lane_000000.png" alt="drawing" width="400"/>

## Download Pretrained VGG-16
The pretrained VGG-16 will be downloaded automatically if it is not located in the *Utils* directory. <br>

In [ ]:
maybe_download_and_extract('http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz')

'''Function for load VGG-16 to tf-session'''
def load_vgg_ckpt(sess):
    ckpt = 'Utils/vgg_16.ckpt'
    variables = slim.get_variables(scope='vgg_16', suffix="weights") + slim.get_variables(scope='vgg_16', suffix="biases")
    init_assign_op, init_feed_dict = slim.assign_from_checkpoint(ckpt, variables)
    sess.run(init_assign_op, init_feed_dict)

## <a name="2"></a> Training FCN model with Pretrained VGG-16

In this section, you will implement FCN model with pretrained VGG-16. <br>

<img src="Utils/FCN.png" alt="drawing" width="600"/>

<b>Complete the code in 'Utils/fcn.py' with reference to above image. <br>
Then, run below cells. (If you don't fill the 'Utils/fcn.py', error will be occured in below cells.)

In [ ]:
def run():
    x_placeholder = tf.placeholder(tf.float32, [None, image_shape[0], image_shape[1], 3])
    y_placeholder = tf.placeholder(tf.float32, [None, image_shape[0], image_shape[1], num_classes])
    lr_placeholder = tf.placeholder(tf.float32)
    is_train_placeholder = tf.placeholder(tf.bool)
    fcn_model = FcnModel(x_placeholder, y_placeholder, is_train_placeholder, num_classes)

    global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
    train_opt = tf.train.AdamOptimizer(lr_placeholder).minimize(fcn_model.loss, global_step)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        load_vgg_ckpt(sess)
        saver = tf.train.Saver()
        writer = tf.summary.FileWriter('graphs/train', sess.graph)

        best_loss = np.inf
        for epoch in range(int(epochs)):
            total_loss_value = 0
            for images, labels in get_batches_fn(int(batch)):
                feed = {x_placeholder: images,
                        y_placeholder: labels,
                        lr_placeholder: lr,
                        is_train_placeholder : True }
            
                _, loss_value, summary_value = sess.run([train_opt, fcn_model.loss, fcn_model.summary],
                                                        feed_dict = feed)
                total_loss_value += loss_value
                
                writer.add_summary(summary_value, sess.run(global_step))
            print("epoch: {}/{}, training loss: {:.2f}".format(epoch+1, int(epochs), total_loss_value))
            if total_loss_value < best_loss:
                saver.save(sess, "models/model.ckpt")
                print("    best model update!!!")
         
        logits = tf.reshape(fcn_model.inference_op, (-1, num_classes))
        save_test_samples(test_output_dir, sess, image_shape, logits, is_train_placeholder, x_placeholder)
 
        
if __name__ == '__main__':
    run()


## Visualize Output Image

Visualize one of the output images. You can see all of them in test_output_dir. (Default directory is './test_run/')

In [ ]:
plt_dir = os.path.join(test_output_dir, 'umm_000011.png')
im = Image.open(plt_dir)
plt.imshow(im)